In [61]:
def get_ali_sell_data(timestamp_gt, timestamp_lt, amount0_gt,url):

    import requests
    import pandas as pd
    from datetime import datetime

    # Define the GraphQL endpoint
    url = url

    def fetch_swaps(last_id=None):
        # GraphQL query with pagination
        query = f"""
        {{
        swaps(
            where: {{token0: "0x6b0b3a982b4634ac68dd83a4dbf02311ce324181", timestamp_gt: {timestamp_gt}, timestamp_lt: {timestamp_lt}, amount0_gt:"0"{', id_gt: "' + last_id + '"' if last_id else ''}}}
            first: 1000
        ) {{
            id
            amount0
            amountUSD
            amount1
            timestamp
        }}
        }}
        """
        
        # Send the request to The Graph
        response = requests.post(url, json={'query': query})
        
        if response.status_code == 200:
            data = response.json()
            return data['data']['swaps']
        else:
            print("Failed to fetch data:", response.status_code, response.text)
            return []

    all_swaps = []
    last_id = None

    # Fetch all the swap data with pagination
    while True:
        swaps = fetch_swaps(last_id)
        if not swaps:
            break
        all_swaps.extend(swaps)
        last_id = swaps[-1]['id']

    # Create a DataFrame from the data
    df = pd.DataFrame(all_swaps)
    df['amount0'] = df['amount0'].astype(float)
    df['amount1'] = df['amount1'].astype(float)
    df['amountUSD'] = df['amountUSD'].astype(float)

    # Convert the timestamp to datetime format
    df['timestamp'] = pd.to_numeric(df['timestamp'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    # Sort the DataFrame by timestamp
    df = df.sort_values(by='timestamp')
    df = df.reset_index(drop=True)

    # Save the DataFrame to a CSV file
    df.to_csv('ali_token_sells.csv', index=False)

    # Summarize the data
    print(f"from {df['timestamp'].iloc[0]} to {df['timestamp'].iloc[-1]}")
    sum_amount0 = df['amount0'].sum()
    print(f"ALI Sell Volume: {sum_amount0} ALI ")
    sum_amountUSD = df['amountUSD'].sum()
    print(f"ALI Sell Volume in USD: {sum_amountUSD} USD\n")


In [63]:
from datetime import datetime

# Function to convert date to Unix timestamp
def date_to_timestamp(date_str, date_format='%d %b %Y %H:%M:%S'):
    dt = datetime.strptime(date_str, date_format)
    return int(dt.timestamp())

# Convert specific dates to Unix timestamps
timestamp_gt = date_to_timestamp('01 Jan 2022 00:00:00')
timestamp_lt = date_to_timestamp('01 Jan 2023 00:00:00')


get_ali_sell_data(timestamp_gt,timestamp_lt,0)

# Convert specific dates to Unix timestamps
timestamp_gt = date_to_timestamp('01 Jan 2023 00:00:00')
timestamp_lt = date_to_timestamp('01 Jan 2024 00:00:00')


get_ali_sell_data(timestamp_gt,timestamp_lt,0)
# Convert specific dates to Unix timestamps

timestamp_gt = date_to_timestamp('01 Jan 2024 00:00:00')
timestamp_lt = date_to_timestamp('24 May 2024 23:59:59')


get_ali_sell_data(timestamp_gt,timestamp_lt,0)

from 2022-04-02 05:38:05 to 2022-12-31 18:29:23
ALI Sell Volume: 7553989.341506237 ALI 
ALI Sell Volume in USD: 235208.84936253756 USD

from 2022-12-31 22:17:47 to 2023-12-31 18:26:11
ALI Sell Volume: 285627867.74036807 ALI 
ALI Sell Volume in USD: 10836990.606254626 USD

from 2023-12-31 21:14:59 to 2024-05-24 11:47:23
ALI Sell Volume: 191061835.8349505 ALI 
ALI Sell Volume in USD: 5481281.873343976 USD



In [ ]:
https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2